## To do
1 - Combining the different Snopes Transformations files into one big CSV

2 - Bring in combined Emergent CSV

3 - Logistic Regression to predict the stances

Part 1 - Combining Snopes

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
#Getting a list of all files which I need to concat
files = glob.glob('mscproject/data/emergent/Snopes_Transformations/Snopes*.csv')

In [3]:
len(files)

262

In [4]:
dfs = []
for i in range(0, 262):
    dfs.append(pd.read_csv(files[i]))

In [5]:
merged = pd.concat(dfs, ignore_index = True)
merged.to_csv('mscproject/data/emergent/Snopes_Transformations/Combined.csv', index = None)

Part 2 - Reading in Emergent features

In [6]:
snopes = pd.read_csv('mscproject/data/emergent/Snopes_Transformations/Combined.csv')
emergent = pd.read_csv('mscproject/data/emergent/Emergent_features_and_labels.csv')

In [7]:
snopes.shape

(46772, 521)

In [8]:
snopes.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,510,511,512,513,514,515,516,517,articleHeadline,claimHeadline
0,0,0.025126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,100 Meijer Coupon - Snopes.com,Meijer is offering 100 off Back to School coup...
1,1,-0.011872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,Fake Meijer 100 back-to-school coupon goes vir...,Meijer is offering 100 off Back to School coup...
2,2,0.109390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,100.0,0.0,100 Meijer Coupon - Hoax - Trendolizer,Meijer is offering 100 off Back to School coup...
3,3,-0.201224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,2.0,0.0,100 OFF Meijer Coupon Deals April 2017 HotDeal...,Meijer is offering 100 off Back to School coup...
4,4,0.016830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,10 off 100 in Visa Gift Cards at Meijer - Freq...,Meijer is offering 100 off Back to School coup...


Part 3 - Getting predictions for the articleHeadlineStances of Snopes articles by using a logistic regression model.
This LR model will be trained on the Emergent features and Emergent labels.

In [9]:
X_train = emergent.loc[:, '0':'517']
y_train = emergent.loc[:, 'articleHeadlineStance']
X_test = snopes.loc[:, '0': '517']

In [10]:
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,508,509,510,511,512,513,514,515,516,517
0,0.025126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0
1,-0.011872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0
2,0.109390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,100.0,0.0
3,-0.201224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,2.0,0.0
4,0.016830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0


** Need to do logistic regression here **

I will use L1 regularization (I read that L1 is better for sparse models)

In [11]:
logr = LogisticRegression(penalty = 'l1')
logr.fit(X_train, y_train)
predicted_stances = logr.predict(X_test)

In [12]:
len(predicted_stances)

46772

Now I will combine the article text and the predicted stances into one csv

In [13]:
claims = pd.DataFrame()
claims['claim'] = snopes['claimHeadline']
claims['articleHeadline'] = snopes['articleHeadline']
claims['predictedStance'] = predicted_stances

In [14]:
for_count = [f for f in predicted_stances if f=='for']
against_count = [f for f in predicted_stances if f=='against']
observing_count = [f for f in predicted_stances if f=='observing']
len(for_count), len(against_count), len(observing_count)

(29535, 7968, 9269)

In [15]:
claims.to_csv('Snopes_articles_with_predicted_stance.csv')

## Seeing if XGBoost gives better predicted stances than Logistic Regression

In [16]:
xgb = xgboost.XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [17]:
preds = xgb.predict(X_test)

In [18]:
for_count = [f for f in preds if f=='for']
against_count = [f for f in preds if f=='against']
observing_count = [f for f in preds if f=='observing']
len(for_count), len(against_count), len(observing_count)

(30712, 7713, 8347)

In [30]:
claims['predictedStance']=preds
claims.to_csv('Snopes_articles_with_predicted_stance_XGB.csv')